In [1]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
    
    # Number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    
    for i in range(num_gpus):
        print(f"\nDetails for GPU {i}:")
        print(f"Name: {torch.cuda.get_device_name(i)}")
        print(f"Total Memory: {torch.cuda.get_device_properties(i).total_memory / (1024**3):.2f} GB")
        print(f"Multi-Processor Count: {torch.cuda.get_device_properties(i).multi_processor_count}")
        print(f"Compute Capability: {torch.cuda.get_device_capability(i)}")
else:
    print("CUDA is not available on this system.")

CUDA is available!
Number of GPUs available: 1

Details for GPU 0:
Name: Tesla V100-SXM2-32GB
Total Memory: 31.73 GB
Multi-Processor Count: 80
Compute Capability: (7, 0)


In [18]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import SwinModel
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from tqdm import tqdm


In [19]:
# Load your data
data = np.load('2023-11-15-cine-myo-masks-and-TOS.npy', allow_pickle=True)

In [20]:
# Custom Dataset Class
class MyDataset(Dataset):
    def __init__(self, data, desired_num_frames=32):
        self.inputs = []
        self.targets = []
        self.desired_num_frames = desired_num_frames
        for item in data:
            mask_volume = item['cine_lv_myo_masks_cropped']  # Shape: (H, W, n_frames)
            tos = item['TOS']  # Shape: (126,)
            self.inputs.append(mask_volume)
            self.targets.append(tos)
            
        self.resize = transforms.Resize((224, 224))
        self.to_tensor = transforms.ToTensor()
        self.to_pil = transforms.ToPILImage()
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        mask_volume = self.inputs[idx]  # Shape: (H, W, n_frames)
        tos = self.targets[idx]  # Shape: (126,)
        
        # Rearrange mask_volume to (n_frames, H, W)
        mask_volume = mask_volume.transpose(2, 0, 1)  # Now shape is (n_frames, H, W)
        
        # Ensure the number of frames is desired_num_frames
        num_frames = mask_volume.shape[0]
        desired_num_frames = self.desired_num_frames
        if num_frames < desired_num_frames:
            # Pad with zeros (blank frames)
            pad_size = desired_num_frames - num_frames
            pad_frames = np.zeros((pad_size, mask_volume.shape[1], mask_volume.shape[2]))
            mask_volume = np.concatenate((mask_volume, pad_frames), axis=0)
        elif num_frames > desired_num_frames:
            # Sample frames evenly
            indices = np.linspace(0, num_frames - 1, desired_num_frames, dtype=int)
            mask_volume = mask_volume[indices]
        # Now mask_volume has shape (desired_num_frames, H, W)
        
        # Process each frame
        frames = []
        for frame in mask_volume:
            frame = frame.astype(np.uint8)  # Convert to uint8
            frame_pil = self.to_pil(frame)
            frame_resized = self.resize(frame_pil)
            frame_tensor = self.to_tensor(frame_resized)
            # Repeat the grayscale channel to make it 3 channels
            frame_tensor = frame_tensor.repeat(3, 1, 1)
            frames.append(frame_tensor)
        
        # Stack frames to create a tensor of shape (desired_num_frames, 3, 224, 224)
        frames_tensor = torch.stack(frames)  # Shape: (desired_num_frames, 3, 224, 224)
        
        # Convert tos to tensor
        tos_tensor = torch.tensor(tos, dtype=torch.float32)
        
        return frames_tensor, tos_tensor

In [21]:
# Split dataset into training and evaluation sets
dataset = MyDataset(data, desired_num_frames=32)
train_size = int(0.8 * len(dataset))
eval_size = len(dataset) - train_size
train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=4, shuffle=False)


In [22]:
# Define the Regression Model
class SwinRegressionModel(nn.Module):
    def __init__(self):
        super(SwinRegressionModel, self).__init__()
        self.swin = SwinModel.from_pretrained('microsoft/swin-base-patch4-window7-224')
        self.config = self.swin.config
        self.fc = nn.Linear(self.config.hidden_size, 126)
        
    def forward(self, x):
        # x: (batch_size, desired_num_frames, 3, 224, 224)
        batch_size, n_frames, C, H, W = x.size()
        x = x.view(batch_size * n_frames, C, H, W)  # Merge batch and frames
        outputs = self.swin(x)
        # outputs.last_hidden_state: (batch_size * n_frames, num_patches, hidden_size)
        # Pool over spatial dimensions (mean pooling)
        hidden_states = outputs.last_hidden_state.mean(dim=1)  # (batch_size * n_frames, hidden_size)
        # Reshape back to (batch_size, n_frames, hidden_size)
        hidden_states = hidden_states.view(batch_size, n_frames, -1)
        # Aggregate over frames (e.g., average)
        aggregated_features = hidden_states.mean(dim=1)  # (batch_size, hidden_size)
        regression_output = self.fc(aggregated_features)  # (batch_size, 126)
        return regression_output

In [23]:
# Initialize Model, Loss Function, and Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SwinRegressionModel().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [24]:
# Training Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    
    progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch+1}", unit="batch")
    for batch_idx, batch in enumerate(progress_bar):
        inputs, targets = batch
        inputs = inputs.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)  # outputs: (batch_size, 126)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        progress_bar.set_postfix({"Batch Loss": loss.item()})
    
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} Completed. Average Training Loss: {avg_loss:.4f}")


Epoch 1/10


Training Epoch 1: 100%|██████████| 26/26 [00:33<00:00,  1.30s/batch, Batch Loss=921]    


Epoch 1 Completed. Average Training Loss: 1129.0148

Epoch 2/10


Training Epoch 2: 100%|██████████| 26/26 [00:33<00:00,  1.30s/batch, Batch Loss=781]    


Epoch 2 Completed. Average Training Loss: 871.4726

Epoch 3/10


...

Epoch 8 Completed. Average Training Loss: 313.7200

Epoch 9/10


Training Epoch 9: 100%|██████████| 26/26 [00:34<00:00,  1.32s/batch, Batch Loss=214]


Epoch 9 Completed. Average Training Loss: 290.4711

Epoch 10/10


Training Epoch 10: 100%|██████████| 26/26 [00:34<00:00,  1.31s/batch, Batch Loss=439]

Epoch 10 Completed. Average Training Loss: 278.2290


In [25]:
# Compute R^2 Score on Evaluation Data
def compute_r2_score(model, dataloader, device):
    model.eval()
    all_targets = []
    all_predictions = []
    
    with torch.no_grad():
        for batch in dataloader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            outputs = model(inputs)
            all_predictions.append(outputs.cpu())
            all_targets.append(targets.cpu())
    
    all_predictions = torch.cat(all_predictions, dim=0)
    all_targets = torch.cat(all_targets, dim=0)
    
    mean_targets = torch.mean(all_targets, dim=0)
    total_sum_of_squares = torch.sum((all_targets - mean_targets) ** 2)
    residual_sum_of_squares = torch.sum((all_targets - all_predictions) ** 2)
    r2 = 1 - (residual_sum_of_squares / total_sum_of_squares)
    
    return r2.item()

In [26]:
# Evaluate the model
final_r2_score = compute_r2_score(model, eval_dataloader, device)
print(f"\nFinal R^2 Score on Evaluation Dataset: {final_r2_score:.4f}")


Final R^2 Score on Evaluation Dataset: -0.0075
